In [1]:
#!pip install py2neo
#!pip install neo4j
#!pip install neo4j
#!pip install neo4jupyter
#!pip install openpyxl --upgrade

#!pip install tf_geometric 

# Course: Recommender Systems/Week 3
#Notebook Description: Practical Exercise (NeuMF part)

*Notebook: Practical_Execrcise_NeuMF_Recommender_System_GitHub-.ipynb*

<a href="https://colab.research.google.com/github/gassaf2/RecommenderSystems/blob/main/week3/Practical_Execrcise_NeuMF_Recommender_System_GitHub-.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NewMF

In [2]:
#!pip install "numpy==2.0.2"
#!pip install pybind11 --upgrade
#!pip install --upgrade torch-scatter torch-sparse torch-geometric
#!pip install "numpy<2.0"

In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

In [4]:
print("A")

A


In [5]:

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Devices:", tf.config.list_physical_devices('GPU'))



Num GPUs Available: 0
GPU Devices: []


In [6]:
import sys
import pandas as pd


#import tensorflow as tf
#tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
#from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split,python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)



In [7]:
df = pd.read_excel('Rec_sys_data.xlsx')

In [8]:

df= df[['CustomerID','StockCode','Quantity','DeliveryDate']]

In [9]:

df["StockCode"] = df["StockCode"].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

In [10]:
df = df.dropna()
print(df.shape)
df

(246706, 4)


,CustomerID,StockCode,Quantity,DeliveryDate
1,17850,71053.0,6,2010-12-02 08:26:00
2,17850,21730.0,6,2010-12-03 08:26:00
4,17850,22752.0,2,2010-12-04 08:26:00
7,17850,22633.0,6,2010-12-04 08:28:00
8,17850,22632.0,6,2010-12-03 08:28:00
...,...,...,...,...
272399,15249,23399.0,12,2011-10-08 11:37:00
272400,15249,22727.0,4,2011-10-08 11:37:00
272401,15249,23434.0,12,2011-10-08 11:37:00
272402,15249,23340.0,12,2011-10-07 11:37:00


In [11]:
#header=["userID", "itemID", "rating", "timestamp"]

df = df.rename(columns={

    'CustomerID':"userID",'StockCode':"itemID",'Quantity':"rating",'DeliveryDate':"timestamp"

})

df["userID"] = df["userID"].astype(int)
df["itemID"] = df["itemID"].astype(int)

In [12]:
train, test = python_chrono_split(df, 0.75)

In [13]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185041 entries, 37126 to 137323
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userID     185041 non-null  int32         
 1   itemID     185041 non-null  int32         
 2   rating     185041 non-null  int64         
 3   timestamp  185041 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(2), int64(1)
memory usage: 5.6 MB


In [15]:
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
#MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 5
BATCH_SIZE = 256

SEED = 42
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


In [16]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

C:\Users\gassaf\Documents\3_Personal_Documents\LAU\Course11_Recommender_Systems\Final_Project\pyenv_310\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [17]:
data

In [18]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 191.1677 seconds for training.


In [19]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 327.8661 seconds for prediction.


In [20]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.028861
NDCG:	0.068375
Precision@K:	0.045769
Recall@K:	0.048933


In [21]:
df_order = pd.read_excel('Rec_sys_data.xlsx', 'order')
df_customer = pd.read_excel('Rec_sys_data.xlsx', 'customer')

In [22]:
df_product = pd.read_excel('Rec_sys_data.xlsx', 'product')

In [23]:
all_predictions = all_predictions[['userID','itemID','prediction']]

In [24]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})

In [25]:
def recommend_product(customer_id):

  print(" \n---------- Top 10 Bought StockCodes -----------\n")


  print(df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(10,'Quantity'))

  top_10_bought = df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(10,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')
    
  print("len",len(top_10_bought))
  print(df_product[df_product.StockCode.isin(top_10_bought.StockCode)]['Product Name'])


  print("\n --------- Top 10 Recommendations ------------ \n")

  print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(10,'probability'))

  recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(10,'probability')

  print('\n-------Product Name of Recommendations ------\n')

  print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


In [26]:
recommend_product(16923)

 
---------- Top 10 Bought StockCodes -----------

        CustomerID StockCode  Quantity
266565       16923     21809       192
89211        16923     22294       120
122279       16923     22294        72
227311       16923     22294        72
148095       16923     22294        48
179098       16923     22294        48
132560       16923     21260        40
141191       16923     22494        40
16095        16923     22969        24
148186       16923     22698        24

-------Product Name of bought StockCodes ------

len 10
468           Window Tint Film Mitsubishi (all doors) DIY
1333    Mitsubishi Spyder AC10812 Northstar White Gall...
1365    Business Essentials 8" x 8" x 5" Corrugated Ma...
1619    Dove Advanced Care Antiperspirant Deodorant Or...
1833    MightySkins Skin Decal Wrap Compatible with HP...
2610    TExES Bilingual Education Supplemental (164) S...
Name: Product Name, dtype: object

 --------- Top 10 Recommendations ------------ 

         CustomerID  StockCode 

In [27]:
recommend_product(17850)

 
---------- Top 10 Bought StockCodes -----------

      CustomerID StockCode  Quantity
285        17850    82494L        12
2629       17850    85123A        12
2634       17850     71053        12
2978       17850     71053        12
2983       17850    85123A        12
3299       17850     71053        12
3301       17850     21068        12
3302       17850    84029G        12
3303       17850    85123A        12
3305       17850     37370        12

-------Product Name of bought StockCodes ------

len 10
135    Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...
162    Heavy Duty Handlebar Motorcycle Mount Holder K...
179           AARCO Enclosed Wall Mounted Bulletin Board
436    MightySkins Skin Decal Wrap Compatible with Ni...
669    3 1/2"W x 20"D x 20"H Funston Craftsman Smooth...
967    Awkward Styles Shamrock Flag St. Patrick's Day...
Name: Product Name, dtype: object

 --------- Top 10 Recommendations ------------ 

         CustomerID  StockCode  probability
8944554       1

In [28]:
recommend_product(14034)

 
---------- Top 10 Bought StockCodes -----------

        CustomerID StockCode  Quantity
175575       14034     22576        48
175581       14034     21810        48
175562       14034     23212        36
175582       14034     22130        30
175563       14034     21824        24
175568       14034     22578        16
175576       14034     22855        12
175580       14034     22599        12
175571       14034     22574        10
175603       14034     23212        10

-------Product Name of bought StockCodes ------

len 10
1266         Artist Watercolor viridian, 5 ml (pack of 2)
1302    6pc Boy Formal Necktie Black & White Suit Set ...
1341    Ebe Reading Glasses Mens Womens Gold Harry Pot...
1785    MightySkins Skin Decal Wrap Compatible with HP...
1791    MightySkins Skin Decal Wrap Compatible with Go...
1913    CafePress - Spider-Man Teacher - Women's Dark ...
1944    3 1/2"W x 20"D x 20"H Funston Craftsman Smooth...
2091    Ecclesiastes 8:15 so I commend the enjoymentÃ¢...

In [29]:
recommend_product(14985)

 
---------- Top 10 Bought StockCodes -----------

        CustomerID StockCode  Quantity
141898       14985     21294        48
141907       14985     22713        24
141912       14985     21034        20
23149        14985     16236        12
141908       14985     15034        12
141897       14985     21746         8
23142        14985     21034         7
23160        14985     21642         7
23115        14985     21644         6
141905       14985     20724         5

-------Product Name of bought StockCodes ------

len 10
215     MightySkins Skin Decal Wrap Compatible with Ap...
566     ML7-6 6V 7AH REPLACEMENT BATTERY FOR UPSONIC U...
883     Seachrome Lifestyle & Wellness Left Hand Zuma ...
1374    MightySkins Skin Decal Wrap Compatible with Sm...
1596    Ebe Reading Glasses Mens Womens Black Blue Ret...
1878    Dr. Comfort Paradise Women's Casual Shoe: 4.5 ...
1910    Urbane Urbane Classic Crossover Scrub Top Scru...
1920    3 1/2"W x 20"D x 20"H Funston Craftsman Smooth...

In [30]:
recommend_product(13137)

 
---------- Top 10 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12
94857        13137     20726        10
94868        13137     20725        10
94875        13137     22662        10
94883        13137     22285        10
94892        13137     22270        10

-------Product Name of bought StockCodes ------

len 10
70      MightySkins Skin Decal Wrap Compatible with Li...
165     Ebe Men Black Rectangle Half Rim Spring Hinge ...
202                 billyboards Porcelain Menu Chalkboard
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
935     MightySkins Skin Decal Wrap Compatible with Ap...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...

# Step 3: Discussion of Results [20 points]

Discuss the performance of the GCNN model compared to the Feedforward NeuMF model. Provide insights on which model performs better and why, based on the evaluation metrics. Consider aspects like Precision@K, Recall@K, and F1 score.

Compare the recommended items for Customer 17850 generated by your model with those recommended by Neo4j. Are there similarities between the two sets of recommendations?

<b> Intepretation about performance metric of GCNN </b> <br>
<b> Discussion about the NeuMF performance metrics:</b>

MAP (0.0289) indicates a relatively low mean average precision, suggesting the model struggles to rank relevant items consistently high.

NDCG (0.0684) shows limited effectiveness in placing relevant recommendations near the top of the list.

Precision@K (0.0458) and Recall@K (0.0489) are also modest, reflecting that only about 4–5% of the top-K recommended items are relevant and that the model retrieves a small fraction of all possible relevant items.

These results suggest room for improvement. You might explore tuning hyperparameters, increasing embedding size, or incorporating additional user/item features to enhance performance.

# n_factor=16

In [31]:
model_nfactor16 = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=16,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

C:\Users\gassaf\Documents\3_Personal_Documents\LAU\Course11_Recommender_Systems\Final_Project\pyenv_310\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [32]:
with Timer() as train_time:
    model_nfactor16.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 197.6129 seconds for training.


In [33]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model_nfactor16.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 328.2233 seconds for prediction.


In [34]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.044282
NDCG:	0.094738
Precision@K:	0.060949
Recall@K:	0.075899


In [35]:
df_order = pd.read_excel('Rec_sys_data.xlsx', 'order')
df_customer = pd.read_excel('Rec_sys_data.xlsx', 'customer')

In [36]:
df_product = pd.read_excel('Rec_sys_data.xlsx', 'product')

In [37]:
all_predictions = all_predictions[['userID','itemID','prediction']]

In [38]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})

# n_factor=32

In [39]:
model_nfactor32 = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=32,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

C:\Users\gassaf\Documents\3_Personal_Documents\LAU\Course11_Recommender_Systems\Final_Project\pyenv_310\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [40]:
with Timer() as train_time:
    model_nfactor32.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 217.2573 seconds for training.


In [41]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model_nfactor32.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 326.0792 seconds for prediction.


In [42]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.053632
NDCG:	0.111667
Precision@K:	0.069525
Recall@K:	0.092856


In [43]:
df_order = pd.read_excel('Rec_sys_data.xlsx', 'order')
df_customer = pd.read_excel('Rec_sys_data.xlsx', 'customer')

In [44]:
df_product = pd.read_excel('Rec_sys_data.xlsx', 'product')

In [45]:
all_predictions = all_predictions[['userID','itemID','prediction']]

In [46]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})